<a href="https://colab.research.google.com/github/NoujoudNader/MatrixMultiplication/blob/main/gcn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import networkx as nx
import matplotlib.pyplot as plt

# Define the Graph Generation Model
class GraphGenerationModel(tf.keras.Model):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GraphGenerationModel, self).__init__()
        self.gcn = tf.keras.Sequential([
            layers.Dense(hidden_dim, activation='relu'),
            layers.Dense(hidden_dim, activation='relu'),
            layers.Dense(output_dim)
        ])

    def call(self, inputs):
        return self.gcn(inputs)


# Define the nodes and edges
nodes = ['A', 'B', 'C', 'D', 'E', 'F']
edges = [('A', 'B', 5), ('B', 'C', 2), ('B', 'D', 3), ('C', 'E', 4), ('C', 'F', 1)]

# Create a mapping from nodes to indices
node_indices = {node: i for i, node in enumerate(nodes)}

# Convert edges and node features to numerical representations
input_edges = edges[:3]

input_edge_indices = np.array([(node_indices[src], node_indices[dst]) for src, dst, _ in input_edges])
input_edge_weights = np.array([weight for _, _, weight in input_edges])

# Filter the nodes to include only those present in the input edges
filtered_nodes = list(set([src for src, _, _ in input_edges] + [dst for _, dst, _ in input_edges]))

# Create node features for the filtered nodes
node_features = np.ones((len(filtered_nodes), 1))  # Example node features

# Define the model and optimizer
model = GraphGenerationModel(input_dim=1, hidden_dim=64, output_dim=1)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
loss_func = tf.keras.losses.MeanSquaredError()

# Train the model
for epoch in range(100):
    with tf.GradientTape() as tape:
        pred_edges = model(node_features)
        pred_output_edges = tf.gather(pred_edges, input_edge_indices[:, 0], axis=1)
        loss = loss_func(input_edge_weights, pred_output_edges)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    # Print loss per epoch
    print(f"Epoch {epoch + 1}: Loss = {loss.numpy()}")

# Generate new edges based on the trained model
num_new_edges = 2
new_edges = []
for _ in range(num_new_edges):
    pred_output_weights = model(node_features)[input_edge_indices[:, 0]]
    max_weight_idx = tf.argmax(pred_output_weights, axis=0).numpy()
    src, dst, _ = input_edges[max_weight_idx]
    new_weight = pred_output_weights[max_weight_idx][0].numpy()
    new_edge = (dst, np.random.choice(nodes), new_weight)  # Randomly select a destination node
    new_edges.append(new_edge)

# Create the complete graph with the input edges and predicted output edges
complete_graph = input_edges + new_edges

# Create a directed graph from the edges
graph = nx.DiGraph()
graph.add_weighted_edges_from(complete_graph)

# Draw the graph as a tree
pos = nx.nx_pydot.graphviz_layout(graph, prog='dot')
nx.draw(graph, pos, with_labels=True, node_color='lightblue', node_size=2000, font_size=12, arrows=True)
edge_labels = nx.get_edge_attributes(graph, 'weight')
nx.draw_networkx_edge_labels(graph, pos, edge_labels=edge_labels, font_color='red')

plt.title("Graph as a Tree")
plt.show()


InvalidArgumentError: ignored